# CoDeepNEAT demo
## CS081 project checkpoint demo
### Harsha Uppli, Alan Zhao, Gabriel Meyer-Lee

The following notebook demonstrates using CoDeepNEAT to solve CIFAR-10

In [1]:
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical
from math import pi, floor
from random import random
from codeepneat import codeepneat, config, population, chromosome, genome, visualize
import pickle
import numpy as np
import keras

Using TensorFlow backend.


### Problem: CIFAR10 data set

Conveniently, it's also built into Keras, which our CoDeepNEAT imiplementation is built off of.

In [10]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = y_train[:,0]
y_test = y_test[:,0]
print("data shapes")
print("  x train:", x_train.shape)
print("  x test:", x_test.shape)
print("  y train:", y_train.shape)
print("  y test:", y_test.shape)
x_train = np.reshape(x_train, (x_train.shape[0], 32, 32, 3)).astype('float32') / 255
x_test = np.reshape(x_train, (x_test.shape[0], 32, 32, 3)).astype('float32') / 255
y_train = keras.utils.np_utils.to_categorical(y_train)
y_test = keras.utils.np_utils.to_categorical(y_test)
data = [x_train, y_train, x_test, y_test]

data shapes
  x train: (50000, 32, 32, 3)
  x test: (10000, 32, 32, 3)
  y train: (50000,)
  y test: (10000,)


ValueError: cannot reshape array of size 153600000 into shape (10000,32,32,3)

### Configuring NEAT

Many of the options and inputs are still handled through the config file. The config file has been shortened considerably as many parameters have been eliminated, although many parameters have also been introduced which could be added.

In [11]:
%%file configCIFAR10
#--- parameters for the robot experiment ---#
[phenotype]
input_nodes         = 32,32,3
output_nodes        = 10
conv                = True

[genetic]
max_fitness_threshold = 1

# Human reasoning
pop_size              = 10
prob_addconv          = 0.9
prob_addlayer         = 0.1
prob_addmodule        = 0.05
elitism               = 1

[genotype compatibility]
compatibility_threshold = 3.0
compatibility_change    = 0.0
excess_coeficient       = 1.0
disjoint_coeficient     = 1.0
connection_coeficient   = 0.4

[species]
species_size        = 10
survival_threshold  = 0.2
old_threshold       = 30
youth_threshold     = 10
old_penalty         = 0.2
youth_boost         = 1.2
max_stagnation      = 15

Overwriting configCIFAR10


### Fitness

For this demonstration we'll be using supervised learning to train the networks produced by CoDeepNEAT on CIFAR-10 and will use their accuracy after 5 epochs as our fitness. CIFAR-10, like MNIST, is a 10 category classification problem.

In [12]:
def fitness(network, data):
    network.fit(data[0], data[1],  epochs=5)
    loss, acc = network.evaluate(data[2], data[3])
    return acc

### Evolution

Evolution with CoDeepNEAT is slightly different than evolution with NEAT. The main difference is coevolution, where we have two separate populations with a hierarchical relationship evolving together.

In [13]:
def evolve(n):
    config.load('configCIFAR10')
    # Create 2 separate populations (size is now defined explicitly, but config file can still be used)
    module_pop = population.Population(40, chromosome.ModuleChromo)
    # As the top hierarchical level, the blueprint population needs to be able to see the module population
    blueprint_pop = population.Population(10, chromosome.BlueprintChromo, module_pop)
    # Most of the actual evolving is now handled outside of the population, by CoDeepNEAT
    # Instead of requiring the user to overwrite the evaluation function, CoDeepNEAT evaluates the populations itself,
    # it simply requires a fitness function for the networks it creates passed in as an argument.
    codeepneat.epoch(n, blueprint_pop, module_pop, 10, fitness, data, save_best=True, name='CIFAR10')
    # It will still stop if fitness surpasses the max_fitness_threshold in config file
    # Plots the evolution of the best/average fitness
    visualize.plot_stats(module_pop.stats, name="CIFAR10mod_")
    visualize.plot_stats(blueprint_pop.stats, name="CIFAR10bp_")
    # Visualizes speciation
    #visualize.plot_species(module_pop.species_log, name="NMISTmod_")
    #visualize.plot_species(blueprint_pop.species_log, name="NMISTbp_")

In [14]:
evolve(5)

-----Generation 0--------
Network 0
Epoch 1/5


InternalError: Blas SGEMM launch failed : m=32768, n=256, k=3
	 [[Node: model_10/conv2d_4/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](_arg_input_8_0_2/_105, conv2d_4/kernel/read)]]
	 [[Node: metrics_2/acc/Mean/_133 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_585_metrics_2/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'model_10/conv2d_4/convolution', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/scratch/knerr/venvs/cs81venv/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance

In [ ]:
def eval_best(model_file):
    config.load('configCIFAR10')
    model = keras.models.load_model(model_file)
    visualize.draw_net(model, "_" + model_file)    
    model.fit(data[0], data[1], epochs=50)
    loss, fitness = model.evaluate(data[2], data[3])
    print("fitness", fitness)

In [ ]:
eval_best("CIFAR10_best_model_0")